In [1]:
%pip install numpy pandas scikit-learn

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/b4/ca/fc1c4f8a2a4693ff437d039acf2dc93a190b9494569fbed246f535c44fc8/numpy-1.26.0-cp39-cp39-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 842.1 kB/s eta 0:00:001m827.1 kB/s eta 0:00:01
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/c5/2f/bf85305b044ddee0ade62c444c7ef551eb423899424b3898d60895d02f63/pandas-2.1.1-cp39-cp39-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/88/a3/023bf03411b8d89954d028dd56dc97f5fa04f18b4cb2acf4315dfb3dfa15/scikit_learn-1.3.1-cp39-cp39-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2023.3-py2.py3

In [2]:
import pandas as pd
# read json into a dataframe
df_idf=pd.read_json("data/stackoverflow-data-idf.json",lines=True)
# print schema
print("Schema:\n\n",df_idf.dtypes)
print("Number of questions,columns=",df_idf.shape)

Schema:

 id                            int64
title                        object
body                         object
answer_count                  int64
comment_count                 int64
creation_date                object
last_activity_date           object
last_editor_display_name     object
owner_display_name           object
owner_user_id               float64
post_type_id                  int64
score                         int64
tags                         object
view_count                    int64
accepted_answer_id          float64
favorite_count              float64
last_edit_date               object
last_editor_user_id         float64
community_owned_date         object
dtype: object
Number of questions,columns= (20000, 19)


In [3]:
import re
def pre_process(text):
	# lowercase
	text= text.lower()
	#remove tags <> using regex
	text = re.sub(r'<.*?>', '', text) 
	# remove special characters and digits
	text= re.sub("(\\d|\\W)+"," ",text)
	return text
df_idf['text'] = df_idf['title'] + df_idf['body']
df_idf['text'] = df_idf['text'].apply(pre_process) #TODO: Make appropriate call
#show the first 'text'
df_idf['text'][2]

'gradle command linei m trying to run a shell script with gradle i currently have something like this def test project tasks create test exec commandline bash c bash c my file dir script sh the problem is that i cannot run this script because i have spaces in my dir name i have tried everything e g commandline bash c bash c my file dir script sh tokenize commandline bash c bash c my file dir script sh commandline bash c new stringbuilder append bash append c my file dir script sh commandline bash c bash c my file dir script sh file dir file c my file dir script sh commandline bash c bash dir getabsolutepath im using windows bit and if i use a path without spaces the script runs perfectly therefore the only issue as i can see is how gradle handles spaces '

In [19]:
from sklearn.feature_extraction.text import CountVectorizer 
import re
def get_stop_words(stop_file_path): 
    """load stop words """
    with open(stop_file_path, 'r', encoding="utf-8") as f: 
        # TODO: Read lines from the file f 
        stopwords = f.readlines()
        # TODO: Strip all stopwords
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)
#load a set of stop words
stopwords=get_stop_words("resources/stopwords.txt") #get the text column
stopwords = list(stopwords)
docs=df_idf['text'].tolist()
print(len(docs))
#create a vocabulary of words, ignore words that appear in 85% of documents, eliminate stop words
cv = CountVectorizer(max_df=0.85,stop_words=stopwords)
# cv= CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector= cv.fit_transform(docs)
word_count_vector.shape

20000


/Users/shreyasv/Downloads/Lab 4-20230923/.venv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  warnings.warn(


(20000, 131802)

In [20]:
# TODO: Previous definition with the parameter max_features=10000 
cv= CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector= cv.fit_transform(docs)
word_count_vector.shape

(20000, 10000)

In [21]:
list(cv.vocabulary_.keys())[:10]

['serializing',
 'private',
 'struct',
 'public',
 'class',
 'contains',
 'properties',
 'string',
 'serialize',
 'attempt']

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [24]:
tfidf_transformer.idf_

array([ 7.37717703,  9.80492526,  9.51724319, ...,  8.82409601,
       10.21039037,  9.51724319])

In [25]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_json("data/stackoverflow-test.json",lines=True) 
df_test['text'] = df_test['title'] + df_test['body']
df_test['text'] =df_test['text'].apply(pre_process) #TODO: Make appropriate call 
# get test docs into a list
docs_test=df_test['text'].tolist()
docs_title=df_test['title'].tolist()
docs_body=df_test['body'].tolist()

In [30]:
def sort_coo(coo_matrix):
	tuples = zip(coo_matrix.col, coo_matrix.data)
	return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
def extract_topn_from_vector(feature_names, sorted_items, topn=10): 
	"""get the feature names and tf-idf score of top n items"""

	#use only topn items from vector
	sorted_items = sorted_items[:topn+1] # TODO: Splice the data 
	score_vals = []
	feature_vals = []
	for idx, score in sorted_items:
		fname = feature_names[idx]
		#keep track of feature name and its corresponding score
		# TODO: Append score rounded off to 3 decimal places, in the score_vals list
		score_vals.append(round(score, 3))
		# TODO: Append fname to the feature_vals list
		feature_vals.append(feature_names[idx])
	#create a tuples of feature,score
	#results = zip(feature_vals,score_vals)
	results= {}
	for idx in range(len(feature_vals)):
		results[feature_vals[idx]]=score_vals[idx] 
	return results

In [31]:
# you only needs to do this once
# TODO: get_feature_names using initialized cv # get the document that we want to extract keywords from
feature_names=cv.get_feature_names_out()
doc=docs_test[0]
#generate tf-idf for the given document using tfidf_transformer
tf_idf_vector = tfidf_transformer.transform(cv.transform([doc]))
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names, sorted_items) # TODO: Pass params
# now print the results
print("\n=====Title=====")
print(docs_title[0])
print("\n=====Body=====")
print(docs_body[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Title=====
Integrate War-Plugin for m2eclipse into Eclipse Project

=====Body=====
<p>I set up a small web project with JSF and Maven. Now I want to deploy on a Tomcat server. Is there a possibility to automate that like a button in Eclipse that automatically deploys the project to Tomcat?</p>

<p>I read about a the <a href="http://maven.apache.org/plugins/maven-war-plugin/" rel="nofollow noreferrer">Maven War Plugin</a> but I couldn't find a tutorial how to integrate that into my process (eclipse/m2eclipse).</p>

<p>Can you link me to help or try to explain it. Thanks.</p>

===Keywords===
eclipse 0.594
war 0.317
integrate 0.282
maven 0.274
tomcat 0.27
plugin 0.215
projecti 0.165
project 0.16
automate 0.158
jsf 0.152
possibility 0.147


In [32]:
# put the common code into several methods
def get_keywords(idx):
	#generate tf-idf for the given document using tfidf_transformer 
	tf_idf_vector= tfidf_transformer.transform(cv.transform([docs_test[idx]]))
	#sort the tf-idf vectors by descending order of scores
	sorted_items=sort_coo(tf_idf_vector.tocoo())
	#extract only the top n; n here is 10
	keywords=extract_topn_from_vector(feature_names, sorted_items) # TODO: Pass params
	return keywords

def print_results(idx,keywords):
	# now print the results
	print("\n=====Title=====")
	print(docs_title[idx])
	print("\n=====Body=====")
	print(docs_body[idx])
	print("\n===Keywords===")
	for k in keywords:
		print(k,keywords[k])

In [33]:
idx=120 
keywords=get_keywords(idx) 
print_results(idx,keywords)


=====Title=====
SQL Import Wizard - Error

=====Body=====
<p>I have a CSV file that I'm trying to import into SQL Management Server Studio.</p>

<p>In Excel, the column giving me trouble looks like this:
<a href="https://i.stack.imgur.com/pm0uS.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/pm0uS.png" alt="enter image description here"></a></p>

<p>Tasks > import data > Flat Source File > select file</p>

<p><a href="https://i.stack.imgur.com/G4b6I.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/G4b6I.png" alt="enter image description here"></a></p>

<p>I set the data type for this column to DT_NUMERIC, adjust the DataScale to 2 in order to get 2 decimal places, but when I click over to Preview, I see that it's clearly not recognizing the numbers appropriately:</p>

<p><a href="https://i.stack.imgur.com/NZhiQ.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/NZhiQ.png" alt="enter image description here"></a></p>

<p>The column ma

In [34]:
#generate tf-idf for all documents in your list. docs_test has 500 documents using tfidf_transformer
# TODO. HINT - Use docs_test for the transform() of cv
tf_idf_vector= tfidf_transformer.transform(cv.transform(docs_test))
results=[]
for i in range(tf_idf_vector.shape[0]):
	# get vector for a single document
	vector = tf_idf_vector[i]
	#sort the tf-idf vector by descending order of scores
	sorted_items=sort_coo(vector.tocoo())
	keywords=extract_topn_from_vector(feature_names, sorted_items)
	results.append(keywords) 
df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
df

,doc,keywords
0,serializing a private struct can it be done i ...,"{'eclipse': 0.594, 'war': 0.317, 'integrate': ..."
1,how do i prevent floated right content from ov...,"{'evaluate': 0.473, 'content': 0.404, 'console..."
2,gradle command linei m trying to run a shell s...,"{'dynamic': 0.41, 'performed': 0.354, 'operati..."
3,loop variable as parameter in asynchronous fun...,"{'image': 0.424, 'jpg': 0.412, 'background': 0..."
4,canot get the href valuehi i need to valid the...,"{'uri': 0.371, 'bitmap': 0.318, 'intent': 0.30..."
...,...,...
495,how to unbind click and click submit button in...,"{'delphi': 0.617, 'compatible': 0.364, 'win': ..."
496,swaggerui auth redirect swaggeruiauth of nulli...,"{'node': 0.548, 'selectsinglenode': 0.303, 'nu..."
497,ssrs value display error for ssrs conditional ...,"{'logo': 0.553, 'step': 0.332, 'triangle': 0.3..."
498,accessing and changing a class instance from a...,"{'length': 0.426, 'ev': 0.415, 'introduce': 0...."


# Exercise

In [42]:
# DATA BLOCK
# Each new line (after '\n') represents a new document
text = '''he really really loves coffee
my sister dislikes coffee
my sister loves tea'''
import math

def main(text):
	# split the text first into lines and then into lists of words 
	docs = [line.split() for line in text.split('\n')]
	N = len(docs)
	# create the vocabulary: the list of words that appear at leastonce
	vocabulary = list(set([word for doc in docs for word in doc]))
	# print(vocabulary)
	df = {}
	tf = {}
	for word in vocabulary:
		# tf: number of occurrences of word w in document divided by document length
		# note: tf[word] will be a list containing the tf of each word for each document
		# for example tf['he'][0] contains the term frequence of the word 'he' in the first document
		tf[word] = [doc.count(word)/len(doc) for doc in docs]
		# df: number of documents containing word w
		df[word] = sum([word in doc for doc in docs])
	# loop through documents to calculate the tf-idf values
	print(tf)
	print(df)
	for doc_index, doc in enumerate(docs): 
		tfidf = []
		print('\nDocument %d:' % (doc_index + 1))
		print(doc)
		for word in vocabulary:
			# print(word, end=' ')
			word_tf = tf[word][doc_index]
			# print(tf, end=' ')
			word_df = df[word]
			# print(word_df, end=' ')
			# TODO (Don’t forget the base for log is 10)
			word_tfidf = word_tf * math.log(N/word_df,10)
			tfidf.append(word_tfidf)
		print(tfidf)
main(text)

{'loves': [0.2, 0.0, 0.25], 'really': [0.4, 0.0, 0.0], 'coffee': [0.2, 0.25, 0.0], 'he': [0.2, 0.0, 0.0], 'dislikes': [0.0, 0.25, 0.0], 'my': [0.0, 0.25, 0.25], 'sister': [0.0, 0.25, 0.25], 'tea': [0.0, 0.0, 0.25]}
{'loves': 2, 'really': 1, 'coffee': 2, 'he': 1, 'dislikes': 1, 'my': 2, 'sister': 2, 'tea': 1}

Document 1:
['he', 'really', 'really', 'loves', 'coffee']
[0.03521825181113625, 0.19084850188786498, 0.03521825181113625, 0.09542425094393249, 0.0, 0.0, 0.0, 0.0]

Document 2:
['my', 'sister', 'dislikes', 'coffee']
[0.0, 0.0, 0.04402281476392031, 0.0, 0.11928031367991561, 0.04402281476392031, 0.04402281476392031, 0.0]

Document 3:
['my', 'sister', 'loves', 'tea']
[0.04402281476392031, 0.0, 0.0, 0.0, 0.0, 0.04402281476392031, 0.04402281476392031, 0.11928031367991561]


In [43]:
!pip freeze > requirements.txt